In [45]:
from scipy.optimize import curve_fit
from numpy import exp, arange


# Lining up peaks before calculating S-Parameters
The following project is an attempt to line up peak centers from different data curves obtained through dopplar broadening experiments. The idea is to correct for detector drift that may have occured between samples. 


The alignment is going to be done by running a gaussian curve fit to find the center of the peak, rather than using the maximum. This is because the curves are not perfectly smooth. In order to run the curve fit, we need to define the gaussian function. 

In [3]:
#gaussian function used for curve fit. 
def gaussian(x, mu, sigma, k, C):
    # simple gaussian
    return k*exp(-(x-mu)**2 / (2 * sigma**2)) + C




The working code will be implemented into the PAPPy program. PAPPy can provide an array for bins, energy, and counts. The returned output should be just the array of the counts because that is the only thing that will be adjusted. This means that after adjustment, the peak will not be at the same energy as it was before. Assuming detector drift is the problem, counts will drift from energy, so the result should be restored to what it would have been if drift had not occured. To make things simple I am going to shift based on index of the array, meaning the bins and energy arrays will not need to be given. 

The energy and counts arrays are actually matrices where each row represents the data from a sample. This means the function will also need to take in the index for the reference sample.


When the curve is shifted, we will lose some information on the side that it is shifted towards, and there will be a gap on the side that it is shifted away from. To make things easy, I am going to copy the index closest to the gap and spread it over. These will be at the tails and their only influence will be on the curve fit for background subtraction, which will be a minimal effect. 

The following is a demonstration of how the shifting can work. 

In [32]:
a0 = [0,1,2,3,4,5,4,3,2,1]
a1 = [0,0,0,1,2,3,4,5,4,3]

idx0 = 5
idx1 = 7

shiftAmount = idx1 - idx0

shifted = a1
if shiftAmount > 0:
    shifted = a1[shiftAmount:]
    for i in range(shiftAmount):
        shifted.append(a1[-1])
elif shiftAmount < 0:
    shifted = []
    for i in range(-1*shiftAmount):
        shifted.append(a1[0])
    shifted += a1[:(shiftAmount)]
else:
    #its the same
    shifted = a1
print(shifted)


[0, 1, 2, 3, 4, 5, 4, 3, 3, 3]


## Defining the main function
The following function is the function that can be inserted into PAPPy.py in the pasphysics.py file. 

In [46]:
def alignPeaks(counts, ref_idx):
    correctedCounts = [] #we will append an array to this array for each sample

    refCounts = counts[ref_idx]

    #find the center of the counts in the energy spectrum.     
    # uses a gaussian fit to find the z-value and the bounds for a 0.5 s-parameter. 
    p0 = [len(refCounts), 15, max(refCounts), max(refCounts)/10]

    #first fit gaussian to find std and mu. 
    params, _ = curve_fit(gaussian, arange(0,len(refCounts)), refCounts, p0=p0)
    
    refCenter = int(params[0]) #The index for where the center should be shifted to. 

    #now using that center, align the other samples
    for sampleCounts in counts:
        if counts.index(sampleCounts) != ref_idx: #don't align if its the reference. 
            #Step 1, find the center of this sample
            p0 = [len(sampleCounts), 15, max(sampleCounts), max(sampleCounts)/10]
            params, _ = curve_fit(gaussian, arange(0,len(sampleCounts)), sampleCounts, p0=p0)
            sampleCenter = int(params[0]) #The index for the sample's center
    
            
            shiftAmount = sampleCenter - refCenter

            shifted = sampleCounts
            if shiftAmount > 0:
                shifted = sampleCounts[shiftAmount:]
                for _ in range(shiftAmount):
                    #fill in the gap created with the last value
                    shifted.append(sampleCounts[-1])
            elif shiftAmount < 0:
                shifted = []
                for _ in range(-1*shiftAmount):
                    #fill in the gap created at the beginning with copies of the first value
                    shifted.append(sampleCounts[0])
                shifted += sampleCounts[:(shiftAmount)]
            else:
                #its the same, so don't shift it
                shifted = sampleCounts

            correctedCounts.append(shifted)
        else:
            #if it is the reference, just add it without adjusting it. 
            correctedCounts.append(sampleCounts)


    return correctedCounts

In [49]:
ref_idx = 0
counts = [[0,0,0,1,2,3,4,5,6,7,8,9,8,7,6,5,4,3,2,1,0,0],
          [0,0,1,2,3,4,5,6,7,8,9,7,6,5,4,3,2,1,0,0,0,0]]

shiftedArrays = alignPeaks(counts, ref_idx)
print(counts[0],shiftedArrays[0])
print(counts[1],shiftedArrays[1])

[0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 0] [0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 0]
[0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 7, 6, 5, 4, 3, 2, 1, 0, 0, 0, 0] [0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 7, 6, 5, 4, 3, 2, 1, 0, 0, 0]


This test seems to show it working pretty well. The upper left array is the reference, and the upper right is also the reference sample (in the shifted array of arrays). The lower arrays are the original test on the left, and the shifted test on the right. It can be seen that the peak of 9 lines up. This is a crude test because the example distribution was not close to gaussian. The next thing to do would be to test it with a real sample. 

It is also interesting to change the ref_idx and see that it does still work, making the second sample the reference. 

## Review
The code seems to work but it aligns the peaks in the cropped window. It would be better if the peak guess could be made off of a 511 keV from the energy array, and the shift was applied to the entire data rather than just the cropped window. This would matter if we wanted to compare two detectors or data taken with two different amplifications. 